# Exploring OpenAI API

## Task 1: Import Required Modules and Packages

This section imports all the necessary libraries and modules that will be used throughout this notebook for interacting with OpenAI's API, data manipulation, file handling, and visualization.

In [ ]:
from openai import OpenAI
import pandas as pd
import requests
from datetime import datetime
from pprint import pprint
import tiktoken
from pypdf import PdfReader
from IPython.display import Image, Markdown, display
import os
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

## Task 2: Set Up API Key

This section initializes the OpenAI client using your API key stored as an environment variable. This client will be used for all interactions with OpenAI's models.

In [ ]:
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

## Task 2.1: Configure Pandas Display Options

This section sets up display options for pandas DataFrames to ensure that content is shown properly without truncation, and defines a helper function to format DataFrames with improved styling.

In [ ]:
pd.set_option('display.max_colwidth', None)


def pp(df):
    return display(df.style.set_properties(subset=['emails'], **{'text-align': 'left', 'white-space': 'pre-wrap', 'width': '900px'}))

## Task 3: Define Folder Paths and Review Data

This section establishes folder paths for storing reviews and generated emails, and defines a list of example customer reviews that will be used for generating response emails.

In [ ]:
# Path to the 'reviews' folder
reviews_folder = "./reviews"

# Path to the 'emails' folder
emails_folder = "./emails"

# List of reviews
reviews = [
    "These hiking boots exceeded my expectations. Great ankle support and waterproofing that actually works when crossing streams. Perfect for day hikes and weekend backpacking trips.",
    "The sci-fi trilogy by Marcus Chen kept me on the edge of my seat! Incredible world-building and character development throughout all three books. Really hoping they adapt this for a streaming series soon!",
    "This coffee maker is remarkably quiet compared to my previous one. Brews quickly and maintains perfect temperature. Definitely recommend for any coffee enthusiast.",
    "The jacket runs slightly large and the material is heavier than what I expected based on the description. Had to return it and size down. The quality seems good but it's not ideal for transitional weather as advertised.",
    "Absolutely impressed with these noise-cancelling headphones. Crystal clear audio without distortion even at high volumes, and the battery lasts for days. Worth every penny.",
    "Purchased this smart watch last month and I'm thoroughly impressed. The fitness tracking is accurate, and the sleep monitoring has been eye-opening. The band is comfortable for all-day wear, though the charging cable is a bit finicky. Overall, it's become an essential part of my daily routine and the battery easily lasts 3-4 days on a single charge."
]

## Task 4: Create Review Files if Not Already Present

This section creates individual text files for each customer review, saving them to the reviews folder with proper error handling to ensure all files are created successfully.

In [ ]:
# Create the review files in the 'emails' folder with error handling
for i, review in enumerate(reviews, start=1):
    file_path = os.path.join(reviews_folder, f"review{i}.txt")
    try:
        with open(file_path, "w", encoding="utf-8") as file:
            file.write(review)
        print(f"✅ Successfully created: {file_path}")
    except OSError as e:
        print(f"❌ Error writing to {file_path}: {e}")

print(
    f"\nAll review files have been created in the '{reviews_folder}' folder.")

## Task 5: Read Reviews from .txt Files

This section reads all the review files from the reviews folder, loads them into a pandas DataFrame for organized processing, and displays the data for verification.

In [ ]:
# Ensure the folder exists before reading files
if not os.path.exists(reviews_folder):
    raise FileNotFoundError(
        f"The folder '{reviews_folder}' does not exist. Please create it and add review files.")

# Read all .txt files from the 'reviews' folder
reviews = []
for filename in sorted(os.listdir(reviews_folder)):
    if filename.endswith(".txt"):
        file_path = os.path.join(reviews_folder, filename)
        with open(file_path, "r", encoding="utf-8") as file:
            reviews.append(file.read().strip())

# Load reviews into a DataFrame
columns = ['reviews', 'emails']
df = pd.DataFrame(columns=columns)
df['reviews'] = reviews

# Display the DataFrame
df.head()

## Task 6: Generate Emails for Reviews

This section uses OpenAI's GPT model to generate professional customer service email responses to each review. The emails are tailored to thank customers for positive feedback and address concerns in a professional manner, then saved to individual text files.

In [ ]:
chat = [{"role": "system",
         "content": "You are a polite customer support representative."}]

postfix = (
    "\n\nWrite a professional email responding to customer reviews.\n"
    "- Thank customers for positive feedback.\n"
    "- Acknowledge and address concerns professionally.\n"
    "- Encourage continued engagement with the brand.\n"
    "- Keep responses brief and highly professional.\n"
    "- Do not offer promotions, discounts, or recommend other products."
)


def getMail(review):
    chat_history = chat.copy()
    chat_history.append({"role": "user", "content": review+postfix})

    reply = client.chat.completions.create(
        model="gpt-3.5-turbo-1106",
        messages=chat_history
    )

    return reply.choices[0].message.content


# Ensure the folder exists
if not os.path.exists(emails_folder):
    os.makedirs(emails_folder)

df['emails'] = df.apply(lambda x: getMail(x.reviews), axis=1)

# Save emails to files
for index, row in df.iterrows():
    response = row['emails']

    # A. Check if the response is OK
    if response:
        print("✅ Response received successfully.")

        # B. Save the content inside reviewx_email.txt
        email_file_path = os.path.join(
            emails_folder, f"review{index+1}_email.txt")
        try:
            with open(email_file_path, "w", encoding="utf-8") as email_file:
                email_file.write(response)
            print(f"✅ Email content saved to: {email_file_path}")
        except OSError as e:
            print(f"❌ Error saving email content to {email_file_path}: {e}")
    else:
        print("❌ No response received from OpenAI.")

pp(df)

## Task 7: Generate Python Code from Natural Language

This section allows you to generate Python code by providing natural language descriptions. It leverages OpenAI's GPT model to translate your instructions into functioning Python code.

In [ ]:
def generate_python_code(description):
    """
    Generate Python code from a natural language description
    
    Args:
        description (str): Natural language description of the code functionality
    
    Returns:
        str: Generated Python code
    """
    system_prompt = """You are an expert Python programmer. 
Your task is to generate clean, efficient, and well-commented Python code 
based on the user's description. Follow these guidelines:

1. Use Pythonic code style and best practices
2. Add helpful comments explaining complex logic
3. Include docstrings for functions and classes
4. Handle potential errors where appropriate
5. Provide efficient solutions with appropriate data structures
6. Only respond with valid Python code, no explanations outside of code comments"""
    
    chat_messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": f"Generate Python code for: {description}"}
    ]
    
    response = client.chat.completions.create(
        model="gpt-3.5-turbo-1106",
        messages=chat_messages
    )
    
    return response.choices[0].message.content

### Example Code Generation Prompts

Here are some example prompts you can use to generate Python code:

In [ ]:
# Example 1: Data Processing
data_processing_prompt = """Create a function that reads a CSV file containing sales data (columns: date, product_id, quantity, price), 
calculates the total revenue per product, and returns the top 5 products by revenue."""

# Example 2: Web Scraping
web_scraping_prompt = """Write a script using BeautifulSoup and requests to scrape a weather website for the 5-day forecast of a given city, 
and save the results to a JSON file."""

# Example 3: Algorithm Implementation
algorithm_prompt = """Implement a binary search tree class with methods for insertion, deletion, search, and in-order traversal."""

# Example 4: Data Visualization
visualization_prompt = """Create a function that takes a pandas DataFrame with columns 'year' and 'temperature', 
and generates a line plot showing temperature trends over time with proper labeling and styling."""

# Example 5: API Integration
api_prompt = """Write a script that uses the requests library to fetch and display the latest news headlines from a News API, 
with error handling and rate limiting."""

# List of example prompts
example_prompts = [
    data_processing_prompt,
    web_scraping_prompt,
    algorithm_prompt,
    visualization_prompt,
    api_prompt
]

### Generate Code from a Example Prompt

In [ ]:
# Select an example prompt (you can change the index to try different examples)
selected_prompt_index = 0  # 0-based index for the example_prompts list
selected_prompt = example_prompts[selected_prompt_index]

print(f"Selected prompt:\n{selected_prompt}\n")
print("Generating code...\n")

# Generate code from the selected prompt
generated_code = generate_python_code(selected_prompt)

print("Generated Python Code:")
print(generated_code)

### Generate Custom Code

In [ ]:
# Enter your custom code generation prompt
custom_prompt = """Create a function that takes a text string and returns the most frequent words and their counts as a dictionary,
ignoring common stop words like 'the', 'a', 'and', etc."""  # Replace with your own description

print(f"Custom prompt:\n{custom_prompt}\n")
print("Generating code...\n")

# Generate code from the custom prompt
custom_generated_code = generate_python_code(custom_prompt)

print("Generated Python Code:")
print(custom_generated_code)

### Execute Generated Code (Optional)

In [ ]:
# Create a new cell and paste the generated code there to test it
# Make sure to replace any input data with actual values for testing

# Example execution cell:
# (Uncomment and paste your generated code below)

# <paste generated code here>

### Save Generated Code to File (Optional)

In [ ]:
def save_code_to_file(code, filename="generated_code.py"):
    """Save generated code to a Python file"""
    try:
        # Create a 'generated_code' folder if it doesn't exist
        code_folder = "./generated_code"
        if not os.path.exists(code_folder):
            os.makedirs(code_folder)
        
        file_path = os.path.join(code_folder, filename)
        
        with open(file_path, "w", encoding="utf-8") as file:
            file.write(code)
            
        print(f"✅ Code successfully saved to: {file_path}")
        return file_path
    except Exception as e:
        print(f"❌ Error saving code to file: {e}")
        return None

# Save the custom generated code
# Uncomment to save:
# saved_file_path = save_code_to_file(custom_generated_code, "word_frequency_counter.py")